In [ ]:
# -*-coding:utf-8 -*
# importer les packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from functools import reduce
from pandas import DataFrame, Series
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', None)


In [ ]:
# Installer le package missingno

!pip install missingno

In [ ]:
import missingno as msno

<b> Analyse de la forme des données <b>

In [ ]:
# Lire le fichier des donneAnalyse de forme des donnéeses  "fr.openfoodfacts.org.products"

df=pd.read_csv("\\Users\\asus\\Desktop\\FORMATION_2122\\PROJET3\\fr.openfoodfacts.org.products.csv",sep='\t')
 

In [ ]:
# Pour voir les valeurs manquantes
df.info()

In [ ]:
df.head(1)

In [ ]:
df.shape

In [ ]:
# Pour voir les types des variables

df.dtypes.value_counts()

In [ ]:
# To display the missing values in the dataframe
msno.bar(df)

In [ ]:

# To display the missing values in the dataframe
msno.matrix(df, sparkline=True, figsize=(10,5), fontsize=12, color=(0.27, 0.52, 1.0));

In [ ]:
# To display percentage of the missing values in the dataframe
null_percent = df.isnull().sum() / df.shape[0]
null_percent.value_counts

In [ ]:
# lire le fichier des donnees  "data-fields"

df_var = pd.read_table("C:\\Users\\asus\\Desktop\\FORMATION_2122\\PROJET3\\data-fields.txt")
df_var

<b> Choix des variables <b>

In [ ]:
# Choisir la liste des variables pertinentes dans les donnees "fr.openfoodfacts.org.products"

var_pert = ['code', 'product_name','countries_fr','ingredients_text', 'additives_n',
            'nutrition_grade_fr', 'pnns_groups_1', 'pnns_groups_2','energy_100g', 
            'omega-3-fat_100g','fat_100g','saturated-fat_100g', 'carbohydrates_100g',  
            'sugars_100g','fiber_100g','proteins_100g', 'salt_100g','sodium_100g', 
            'vitamin-a_100g', 'vitamin-c_100g', 'nutrition-score-fr_100g']

In [ ]:
len(var_pert)

In [ ]:
# filtrer df pour obtenir un dataframe dont les variables sont dans la liste 'var_pert'

df1 = df[var_pert]

In [ ]:
df1.set_index("code", inplace=True)

In [ ]:
df1.head(1)

In [ ]:
df1.shape

In [ ]:
# filtrer le dataset  df  pour ne garder que les produits francais

df2 = df1.loc[df1['countries_fr']=='France']


In [ ]:
df2.head()

In [ ]:
df2.shape

In [ ]:
# the number of unique entries
# To display number of different values of a variable
df2.nunique().sort_values() 

In [ ]:
# Pourcentage de remplissage des variables

df2.notnull().sum()/df2.shape[0]

In [ ]:
# Number of duplicate values  
df2.duplicated(subset=None, keep=False).sum()

In [ ]:
# Suppression des valeurs dupliquees

df2.drop_duplicates(keep=False,inplace=True)



In [ ]:
df2.shape

In [ ]:
df2.dtypes

In [ ]:
# Pourcentages de repartition des variables dans df2
df2.dtypes.value_counts().plot.pie(autopct='%1.1f%%')

In [ ]:
# Remettre le "code" comme colonne

df2.reset_index("code", inplace=True)

In [ ]:
# Histogramme de distribution des variables

plt.figure(figsize=(4,5))
sns.histplot(df2.dtypes.astype('string'), color = 'lightblue')

In [ ]:
df2

In [ ]:
# Les valeurs manquantes des differentes variables

figure = plt.figure(figsize = (10,3))
sns.set_context('poster', font_scale = 1)
sns.heatmap(df2.isna(), cmap = "Blues_r")

In [ ]:
# Nombre de valeurs manquantes par variable
df2.isna().sum()

In [ ]:
df2.shape

In [ ]:
# Filtrage du dataset df_serving pour que 'product_name' soit different de NaN

df2 = df2[df2['product_name'].notnull()]


In [ ]:
len(df2)

 <b> Valeurs aberrantes des variables <b>

In [ ]:
# Valeurs aberrantes pour la variable 'omega-3-fat_100g '  

df2[df2['omega-3-fat_100g']>100].index.tolist()

In [ ]:
 # les colonnes du dataset dont les valeurs de la variable 'omega-3-fat_100g' ne sont pas NaN      

omega_selected_rows = df2[~df2['omega-3-fat_100g'].isna()]

In [ ]:
omega_selected_rows.shape

In [ ]:
# get names of indexes for which column 'fat_100g'>100 or 'fat_100g'<0
# Valeurs aberrantes de la variable 'fat_100g'

index_fat = df2[(df2['fat_100g']>100) | (df2['fat_100g']<0)].index 

In [ ]:
index_fat

In [ ]:
# get names of indexes for which column 'saturated-fat_100g'>100 or 'saturated-fat_100g'<0
# Valeurs aberrantes de la variable 'saturated-fat_100g'

index_Sfat = df2[(df2['saturated-fat_100g']>100) | (df2['saturated-fat_100g']<0)].index

In [ ]:
index_Sfat

In [ ]:
# get names of indexes for which column 'saturated-fat_100g'>100 or 'saturated-fat_100g'<0
# Valeurs aberrantes de la variable 'carbohydrates_100g'

index_carbo = df2[(df2['carbohydrates_100g']>100) | (df2['carbohydrates_100g']<0)].index

In [ ]:
index_carbo

In [ ]:
# get names of indexes for which column 'sugars_100g'>100 or 'sugars_100g'<0
# Les valeurs aberrantes de la variable 'sugars_100g'

index_sugar = df2[(df2['sugars_100g']>100) | (df2['sugars_100g']<0)].index


In [ ]:
index_sugar

In [ ]:
# get names of indexes for which column 'fiber_100g'>100 or 'fiber_100g'<0
# Les valeurs aberrantes de la variable 'fiber_100g'

index_fiber = df2[(df2['fiber_100g']>100) | (df2['fiber_100g']<0)].index
len(index_fiber) 

In [ ]:
# Les valeurs aberrantes de la variable proteins_100g'

Protein_aber = df2[(df2['proteins_100g']>100) | (df2['proteins_100g']<0)].index.tolist()
len(Protein_aber)

In [ ]:
# get names of indexes for which column 'salt_100g'>100 or 'salt_100g'<0
# Les valeurs aberrantes de la variable 'salt_100g'

index_salt = df2[(df2['salt_100g']>100) | (df2['salt_100g']<0)].index
len(index_salt)

In [ ]:
index_salt

In [ ]:
# Les valeurs aberrantes de la variable 'sodium_100g'

index_sodium = df2[(df2['sodium_100g']>100) | (df2['sodium_100g']<0)].index.tolist()
len(index_sodium)


In [ ]:
# Les valeurs aberrantes de la variable 'vitamin-a_100g'

index_vitamin_a = df2[(df2['vitamin-a_100g']>100) | (df2['vitamin-a_100g']<0)].index.tolist()
len(index_vitamin_a)

In [ ]:
index_vitamin_c = df2[(df2['vitamin-c_100g']>100) | (df2['vitamin-c_100g']<0)].index.tolist()
len(index_vitamin_c)


In [ ]:
index_energy_100g = df2[(df2['energy_100g']>13600) | (df2['energy_100g']<0)].index
len(index_energy_100g)

In [ ]:
index_energy_100g

In [ ]:
df2.notnull().sum().sum()

In [ ]:
#Suppression des valeurs aberrantes des variables 'fat_100g' et 'saturated-fat_100g'

code_aber1 = index_fat 
df2 = df2[~df2['code'].isin(code_aber1)]

In [ ]:
df2.shape

In [ ]:
# Remplacement des valeurs aberrantes des variables 'carbohydrates_100g' , 'sugars_100g' , 'fiber_100g' et 'salt_100g' par valeur max

df2.loc[index_carbo, 'carbohydrates_100g'] =  df2['carbohydrates_100g'].max()

df2.loc[index_sugar, 'sugars_100g'] =  df2['sugars_100g'].max()
df2.loc[index_sugar, 'fiber_100g'] =  df2['fiber_100g'].max()
df2.loc[index_salt, 'salt_100g'] =  df2['salt_100g'].max()

In [ ]:
df2.shape

In [ ]:
# Remplacement des valeurs aberrantes de la variable  'energy_100g' par valeur NaN

df2.loc[(df2['energy_100g']>13600) | (df2['energy_100g']<0), 'energy_100g']=np.nan

In [ ]:
# Nombre de valeurs manquantes dans le dataset  df2

df2.isna().sum().sum()

In [ ]:
df2.info()

<b> Valeur manquantes <b>

In [ ]:
# Distribution des variables   'additives_n', 'energy_100g', 'omega-3-fat_100g' 
#'vitamin-a_100g', 'vitamin-c_100g'

list_var1=['additives_n', 'energy_100g', 'omega-3-fat_100g','vitamin-a_100g', 'vitamin-c_100g']

df3=df2[list_var1]

np.random.seed(0)
ind = np.random.randint(len(df3), size= (len(df3)//2))
df4 = df3.iloc[ind,:]
df4 = df4.select_dtypes('float')

for col in df4.columns:
    
    plt.figure()
    sns.set_context('poster', font_scale = 1)
    sns.displot(data=df4[col].dropna(), bins = 10, kde=True,  aspect = 2)
    #sns.displot(data=df4[col].dropna(), bins = 10, kde=False, color='deepskyblue', aspect = 2, linewidth=0)

plt.show()
    
    

In [ ]:
df3.shape

In [ ]:
df4.info()

In [ ]:
df4['additives_n'].max()

In [ ]:
len(ind)

In [ ]:
df3.shape

In [ ]:
# Remplacer les valeurs manquantes NaN de 'additives_n', 'energy_100g', 'omega-3-fat_100g' 
#'vitamin-a_100g', 'vitamin-c_100g', par des zeros.

list_var1=['additives_n', 'energy_100g', 'omega-3-fat_100g','vitamin-a_100g', 'vitamin-c_100g']
for var1 in list_var1:
    
    df2[var1] = df2[var1].fillna(0)



In [ ]:
# Distribution des variables 'fat_100g','saturated-fat_100g','proteins_100g','fiber_100g','nutrition-score-fr_100g'
list_var2 = ['fat_100g','saturated-fat_100g','proteins_100g','fiber_100g','nutrition-score-fr_100g']

df5=df2[list_var2]

np.random.seed(0)
ind = np.random.randint(len(df5), size= (len(df5)//2))
df6 = df5.iloc[ind,:]
df6 = df6.select_dtypes('float')

for col in df6.columns:
    
    plt.figure()
    sns.set_context('poster', font_scale = 1)
    #sns.displot(data=df6[col].dropna(), fill=True, kde=True, aspect = 2)
    sns.displot(data=df6[col].dropna(), bins = 10, kde=True,  aspect = 2)
    plt.show()

In [ ]:
# Remplacer des valeurs manquantes NaN de certaines variables par la mediane 
list_var2 = ['fat_100g','saturated-fat_100g','proteins_100g','fiber_100g','nutrition-score-fr_100g']

for var2 in list_var2:

    df2[var2] = df2[var2].fillna(df2[var2].median())


  

In [ ]:
df2['fat_100g']

In [ ]:
# Distribution des variables de 'carbohydrates_100g', 'sugars_100g', 'salt_100g','sodium_100g'

list_var3=['carbohydrates_100g', 'sugars_100g', 'salt_100g','sodium_100g']

df9=df2[list_var3]

np.random.seed(0)
ind = np.random.randint(len(df9), size= (len(df9)//2))
df10 = df9.iloc[ind,:]
df10 = df10.select_dtypes('float')

for col in df10.columns:
    
    plt.figure()
    sns.set_context('poster', font_scale = 1)
    #sns.displot(data=df10[col].dropna(), color='deepskyblue', kde=True, fill=True, common_norm=False, linewidth=0)
    sns.displot(data=df10[col].dropna(), bins = 10, kde=True,  aspect = 2)
   # sns.displot(data=df10[col].dropna(), kde=True,  aspect = 2)
    plt.show()

In [ ]:
# Remplacer les valeur manquantes NaN  de 'carbohydrates_100g', 'sugars_100g', 'salt_100g','sodium_100g'
# en utilisant IterativeImputer

# Copy the data
df9_impute_it = df9.copy(deep=True)

# Inititialization
ii_imp = IterativeImputer(estimator=ExtraTreesRegressor(), max_iter=10, random_state=0)

# Tranform
df9_impute_it.loc[:, :] = ii_imp.fit_transform(df9_impute_it)


In [ ]:
df9 = df9_impute_it
df2[list_var3]=df9

In [ ]:
df9_impute_it.isna().sum()

In [ ]:
df2[list_var3].info()

In [ ]:
df2.info()

In [ ]:
print(df2.isnull().values.any())

In [ ]:
df2.isnull().sum()

In [ ]:
df2['pnns_groups_1' ]

In [ ]:
#Check Column Contains a Value in DataFrame
print('Sugary snacks' in set(df2['pnns_groups_1']))

In [ ]:
print('sugary-snacks' in set(df2['pnns_groups_1']))

In [ ]:
# Eliminer une des variables sugary-Snack et Sugary Snack dans pnns_group1
# En utilisant la method 'replace'
df2['pnns_groups_1'] = df2['pnns_groups_1'].replace(['sugary-snacks'],'Sugary snacks')

In [ ]:
print('sugary-snacks' in set(df2['pnns_groups_1']))

In [ ]:
print('Sugary snacks' in set(df2['pnns_groups_1']))

In [ ]:
# Eliminer une des variables oereals-and-potatoes et Cereals-and-potatoes dans pnns_group1
# En utilisant la method 'replace'
df2['pnns_groups_1'] = df2['pnns_groups_1'].replace(['oereals-and-potatoes'],'Cereals and potatoes')

In [ ]:
print('oereals-and-potatoes' in set(df2['pnns_groups_1']))

In [ ]:
df2.shape

In [ ]:
df2['pnns_groups_1']

In [ ]:
df2.to_csv('df2.csv')